In [1]:
from firecrawl import FirecrawlApp, ScrapeOptions
from dotenv import load_dotenv
import os

## Firecrawl Tool

In [2]:
class FirecrawlTool:
    def __init__(self, api_key, instruction: str, template: "str"):
        self.app = FirecrawlApp(api_key=api_key)
        self.instruction = instruction
        self.template = template

    def search(self, search) -> str:
        """
        Hace una busqueda en el sitio web de ChileAtiende y devuelve el contenido en formato Markdown.
        Args:
            search (str): La consulta de búsqueda que se desea realizar.
        """
        response = ""
        finished = False
        trials = 0
        limit = 2
        while not finished and trials < 2:
            trials += 1
            try:
                self.search_result = self.app.search(
                    query = self.instruction + search,
                    limit=limit,
                    country="cl",
                    lang="es",
                    scrape_options=ScrapeOptions(formats=["markdown", "links"])
                )
                if len(self.search_result.data) > 0:
                    # Filtra los resultados para obtener solo los que son fichas de ChileAtiende
                    filtered_results = [
                        result for result in self.search_result.data
                        if result["url"].startswith("https://www.chileatiende.gob.cl/fichas") and not result["url"].endswith("pdf")
                    ]

                    if len(filtered_results) > 0:
                        # Devuelve el contenido en Markdown del primer resultado filtrado
                        for num, result in enumerate(filtered_results, start=1):
                            response += self.template.format(
                                result_number = num,
                                page_title=result["title"],
                                page_url=result["url"],
                                page_content=result["markdown"]
                            )
                        return response
                    else:
                        limit += 2
                        continue
                else:
                    limit += 2
                    if trials == 2:
                        return "Error: No se encontraron resultados."
                    return "Error: No se pudo obtener una respuesta razonable."
            except Exception as e:
                return f"Error al usar Firecrawl: {str(e)}"

In [3]:
FIRECRAWL_INSTRUCTION = "ChileAtiende: " 
FIRECRAWL_SEARCH = "Quiero saber como renovar mi licencia de conducir"
FIRECRAWL_TEMPLATE = """
# Resultado N°{result_number}

## Nombre de la página: 
"{page_title}"

## URL: 
{page_url}

## Contenido: 
{page_content}

"""
# Inicializa la herramienta Firecrawl
firecrawl_tool = FirecrawlTool(
    api_key=os.getenv("FIRECRAWL_API_KEY"),
    instruction=FIRECRAWL_INSTRUCTION,
    template=FIRECRAWL_TEMPLATE
)

search_example = firecrawl_tool.search(FIRECRAWL_SEARCH)
print(search_example)


# Resultado N°1

## Nombre de la página: 
"Licencias de conducir - ChileAtiende"

## URL: 
https://www.chileatiende.gob.cl/fichas/20592-licencias-de-conducir

## Contenido: 
1. [Inicio](https://www.chileatiende.gob.cl/)
2. [ChileAtiende](https://www.chileatiende.gob.cl/instituciones/ZZ001)

# Licencias de conducir

Última actualización: 20 de junio, 2023

A+

A-


## Descripción

La **licencia de conducir es un documento que permite la conducción de vehículos**. En Chile, existen distintas clases de licencia, las que se obtienen en la Dirección de Tránsito Municipal, rindiendo un examen teórico y práctico.

El Ministerio de Transporte y Telecomunicaciones, a través de Comisión Nacional de Seguridad de Tránsito (Conaset), reformó el sistema de otorgamiento de licencias de conducir, implementado un nuevo **examen práctico** para [clase B](https://www.chileatiende.gob.cl/fichas/24065-licencia-de-conducir-no-profesional-clase-b) y para la obtención de la [licencia clase C](https://mejores

In [4]:
AGENT_INSTRUCTIONS = """
**Actúa como un asistente virtual experto en atención ciudadana del Gobierno de Chile. Has trabajado durante 20 años ayudando a personas —especialmente adultos mayores— a entender y realizar trámites públicos de forma clara, amable y eficiente. Eres paciente, empático, y siempre entregas información precisa, detallada y en un lenguaje sencillo.**

Tu objetivo es ayudar al usuario a encontrar respuestas claras sobre trámites y servicios disponibles en el sitio web oficial [ChileAtiende](https://www.chileatiende.gob.cl/). Cuentas con una herramienta que, al recibir una consulta, realiza una búsqueda en el sitio y entrega una respuesta en formato markdown. Cada respuesta incluye:

- 📄 **Nombre de la página de origen**
- 🔗 **Enlace directo a la fuente**
- 📘 **Contenido principal de la página** (explicado de forma comprensible para personas mayores)
- 🧭 **Referencia con formato de cita**, al estilo: `respuesta[^1]` y al final: `[^1]: https://...`

### Sigue estos pasos:

1. **Analiza la consulta del usuario** y asegúrate de comprender qué trámite o información desea conocer.
2. **Realiza una búsqueda con la herramienta provista** que recibe una consulta entrega resultados desde ChileAtiende en formato markdown.
3. **Reescribe la información** en un tono muy amable, cercano y sin tecnicismos, pensando que estás ayudando a una persona mayor que no está familiarizada con procesos digitales.
4. **Incluye el nombre de la página**, el **enlace web** como referencia, y presenta el **contenido en un formato claro y estructurado**, usando subtítulos y viñetas si es necesario.
5. Finaliza siempre con una nota de apoyo, ofreciendo continuar la ayuda si lo necesita.

📌 **Ejemplo del formato de respuesta esperado**:

---

**Trámite: Renovación de Cédula de Identidad**

Para renovar tu cédula de identidad, debes agendar una hora en el Registro Civil. Puedes hacerlo de forma presencial o en línea si tienes ClaveÚnica. Este proceso es muy importante, sobre todo si tu carnet ya está vencido o por vencer[^1].

- **Dónde se hace:** Registro Civil (presencial) o sitio web
- **Requisitos:** Presentar tu cédula vencida. En caso de extravío, debes informar eso.
- **Costo:** $3.820 para chilenos, $4.270 para extranjeros
- **Tiempo estimado:** 7 a 10 días hábiles

[^1]: https://www.chileatiende.gob.cl/fichas/23456-renovacion-cedula-de-identidad
"""

In [5]:
from agno.agent import Agent
from agno.models.google import Gemini

model = Gemini(
    api_key=os.getenv("GOOGLE_API_KEY"),
    id="gemini-2.0-flash", # Opcional, Gemini usa un modelo por defecto si no se especifica
)

agent = Agent(
    model=model, # Opcional, Agno usa un modelo por defecto si no se especifica
    tools=[
        firecrawl_tool.search, # Añade la función de scrape de Firecrawl como herramienta
    ],
    instructions=AGENT_INSTRUCTIONS,
    show_tool_calls=True # Muestra las llamadas a herramientas en la consola
)

In [6]:
agent.print_response(
    "¿Cómo puedo renovar mi licencia de conducir?"
)

/home/vscode/.local/lib/python3.11/site-packages/google/genai/_common.py:240: UserWarning:  is not a valid Type
  warnings.warn(f"{value} is not a valid {cls.__name__}")

Output()

ERROR    Error from Gemini API: 400 INVALID_ARGUMENT. {'error': {'code': 400, 'message': 'Invalid value at         
         \'tools[0].function_declarations[0].parameters.properties[0].value.type\'                                 
         (type.googleapis.com/google.ai.generativelanguage.v1beta.Type), ""', 'status': 'INVALID_ARGUMENT',        
         'details': [{'@type': 'type.googleapis.com/google.rpc.BadRequest', 'fieldViolations': [{'field':          
         'tools[0].function_declarations[0].parameters.properties[0].value.type', 'description': 'Invalid value at 
         \'tools[0].function_declarations[0].parameters.properties[0].value.type\'                                 
         (type.googleapis.com/google.ai.generativelanguage.v1beta.Type), ""'}]}]}}

WARNING  Attempt 1/1 failed: <Response [400 Bad Request]>

ERROR    Failed after 1 attempts. Last error using Gemini(gemini-2.0-flash)

ModelProviderError: <Response [400 Bad Request]>